# Two-particle self consistency (TPSC)

Now, we use the Lindhard function to solve the TPSC approximation explained in the lecture notes
of André-Marie Tremblay and in the following [review](https://arxiv.org/abs/1107.1534). 

In TPSC, the vertices for charge and spin fluctuations are different but are still local, i.e. momentum and frequency independent like the bare $U$ in the Hubbard model. This allows conservations laws and the Pauli principle to be satisfied. 

More specifically, the spin and charge susceptibilities are 

$$ \chi_{sp}(\mathbf{q}, iq_n) \equiv \frac{\chi_0(\mathbf{q}, iq_n)}{1 - \frac{U_{sp}}{2} \chi_0(\mathbf{q}, iq_n)} $$

$$ \chi_{ch}(\mathbf{q}, iq_n) \equiv \frac{\chi_0(\mathbf{q}, iq_n)}{1 + \frac{U_{ch}}{2} \chi_0(\mathbf{q}, iq_n)} $$

The sum over all momenta and frequencies of the spin susceptibility gives the equal-time equal-position correlation function. This gives the sum-rule  

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},iq_n} \chi_{sp}(\mathbf{q}, iq_n)=\left< (n_\uparrow - n_\downarrow)^2\right>=n-2\left< n_\uparrow n_\downarrow\right>
\end{equation}

because the Pauli principle requires that $\left< n_\uparrow^2\right>=\left< n_\uparrow\right>$ since the occupation number on a site is either 0 or 1.

Substituting the TPSC value of the spin susceptibility, double occupancy and $U_{sp}$ can be determined from

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},iq_n} \frac{\chi_0(\mathbf{q},iq_n)}{1-\frac{U_{sp}}{2}\chi_0(\mathbf{q},iq_n)}=n-2\left< n_\uparrow n_\downarrow\right>
\end{equation}

with the ansatz

\begin{equation}
U_{sp}\left<n_\uparrow\right> \left<n_\downarrow\right>=U\left<n_\uparrow n_\downarrow\right>.
\end{equation}

Given the double occupancy, the charge vertex can then be obtained from

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},iq_n} \frac{\chi_0(\mathbf{q},iq_n)}{1+\frac{U_{cn}}{2}\chi_0(\mathbf{q},iq_n)}=n+2\left< n_\uparrow n_\downarrow\right>-n^2.
\end{equation}

Note that the sums over $\mathbf{q}$ and $i\omega_n$ can be interpreted as a trace.

In [ ]:
# Imports 
%matplotlib inline
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, Gf, MeshProduct, inverse
from pytriqs.archive import HDFArchive
from pytriqs.plot.mpl_interface import oplot
import numpy as np
from math import cos, pi

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,9) # set default size for all figuresfrom pytriqs.archive import HDFArchivefrom pytriqs.gf import Gf, inverse

from scipy.optimize import fsolve, brentq

We reload the Lindhard susceptibility from the archive

In [ ]:
#reload chi0
with HDFArchive("tpsc.h5", 'r') as R:
    chi0_kw = R['chi0_kw']

## Behaviour of $U_{sp}$ and $U_{ch}$

The aim of this exercice is to study the behaviour of $U_{sp}$ and $U_{ch}$ 
as we vary $U$. This will require that you solve the TPSC equations
described above. Ultimately you should try to reproduce the following figure
taken from  the __[paper](https://jp1.journaldephysique.org/articles/jp1/abs/1997/11/jp1v7p1309/jp1v7p1309.html)__

<img src="./img/Fig2.png" alt="Drawing" style="width: 250px;"/>

### Exercice 1:

Define a function `chi_rpa(chi0, U)` taking two arguments and that
computes the following quantity:

\begin{equation}
  \chi_{rpa}(\mathbf{q}, iq_n) \equiv \frac{\chi_0(\mathbf{q}, iq_n)}
    {1 - \frac{U}{2} \chi_0(\mathbf{q}, iq_n)}
\end{equation}

This function will be useful to obtain both the spin and the charge
susceptibilities. Indeed we have that

\begin{equation}
 \chi_{sp} = \chi_{rpa}(U_{sp}) \qquad \chi_{ch} = \chi_{rpa}(-U_{ch})
\end{equation}

In [ ]:
def chi_rpa(chi0, U):
    """Compute chi_rpa from chi"""
    return chi0 * inverse(1 - 0.5 * U * chi0)

### Exercice 2:

You will now solve the TPSC equation for $U_{sp}$.

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},iq_n} \chi_{sp}(\mathbf{q},iq_n) = n-2\left< n_\uparrow n_\downarrow\right> = n - 2
\frac{U_{sp}}{U} \left<n_\uparrow\right> \left<n_\downarrow\right>
\end{equation}

We will focus on the case with $n=1$.

* Convince yourself that this equation always has exactly one solution for $U_{sp}$.
* In what interval does $U_{sp}$ take its values?

In order to solve the equation you can use the scipy function `brentq` that finds
the zero of a function in a given interval. Your task is to complete the code below
by:

* Defining the function `Usp_root` whose zero is the solution of the TPSC
  equation above.
* Defining the interval $[a,b]$ where `brentq` has to look for zeroes

In order to help you, we provide a function `trace_chi` that computes
the sum over momenta and Matsubara frequencies of a susceptibility.
You can check your code for $U=2$. For this value, you should get
$U_{sp} \simeq 1.52$

In [ ]:
def trace_chi(chi):
    """Given chi_kw, it computes sum_k sum_\nu chi(k,\nu)""" 
    kmesh, wmesh = chi.mesh.components
    trace = chi.data.sum() / len(kmesh) / wmesh.beta
    return trace.real

def Usp_root(Usp, chi0, n, U):
    """Sets the self-consistency for U_sp as the problem of finding roots"""
    tr_chi_sp = trace_chi(chi_rpa(chi0, U=Usp))
    diff = tr_chi_sp + 0.5 * Usp/U * n**2 - n
    return diff

U = 2.0
n = 1.0
a = 0
b = 2/chi0_kw([np.pi,np.pi,0],0).real
Usp = brentq(Usp_root, a, b, args=(chi0_kw, n, U), xtol=1e-2)
print Usp

### Exercice 3:

You can now solve the TPSC equation for $U_{ch}$:

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},iq_n} \chi_{ch}(\mathbf{q},iq_n)
=n+2\left< n_\uparrow n_\downarrow\right>-n^2
= n+2 \frac{U_{sp}}{U} \left<n_\uparrow\right> \left<n_\downarrow\right>
- n^2
\end{equation}

For $U=2$, you should find $U_{ch} \simeq 3.37$.

In [ ]:
def Uch_root(Uch, chi0, n, U, docc):
    tr_chi = trace_chi(chi_rpa(chi0, U=-Uch))
    diff = tr_chi - 2 * docc - n + n**2
    return diff

docc = 0.25 * Usp / U * n**2
a = 0
b = 100
Uch = brentq(Uch_root, a, b, args=(chi0_kw, n, U, docc), xtol=1e-2)
print Uch

### Exercice 4:

Putting together the codes that you wrote above, you can now define
a function `solve_tpsc(chi0, U, n)` that solves the TPSC equations
for a given value of $U$ and $n$ and returns $U_{sp}$ and $U_{ch}$.

In [ ]:
def solve_tpsc(chi0, U, n):
    Uc = 2/chi0([np.pi,np.pi,0],0).real
    Usp = brentq(Usp_root, 0, Uc, args=(chi0, n, U), xtol=1e-2)
    docc = 0.25 * Usp / U * n**2
    Uch = brentq(Uch_root, 0, 100, args=(chi0, n, U, docc), xtol=1e-2)
    return Usp, Uch, docc, Uc

print solve_tpsc(chi0_kw, 2., 1.)

### Exercice 5:

Scan different values of $U$ between 0.3 and 5 and make a plot
to see if it compares to the paper figure shown above. The density
is always $n=1$.

In [ ]:
n = 1.0

# Initializes a table to store the results
U_vec = np.concatenate((np.arange(0.3, 1., 0.2), np.arange(1., 6., 1.)))    
Usp_vec, Uch_vec, docc_vec = [np.zeros_like(U_vec) for x in xrange(3)]

# Printing the header of the table
print ''.join('| %-11s' % s for s in ['n', 'U', 'Usp', 'Uch', 'docc']), '|'
print '-'*67

# Loop over the different values of bare U
for idx, U in enumerate(U_vec):
    Usp, Uch, docc, Uc = solve_tpsc(chi0_kw, U, n)
    Usp_vec[idx], Uch_vec[idx], docc_vec[idx] = Usp, Uch, docc
    print ''.join('| %4.4E ' % x for x in [n, U, Usp, Uch, docc]), '|'

In [ ]:
from matplotlib.image import imread

# Some manual adjustements here to overlay the original figure. May change from one
# machine to another
im = imread("img/Fig2.png")
plt.imshow(im, extent=(-0.45, 5.35, -3, 20.5), aspect='auto')
plt.plot([0,5,5,0,0],[0,0,20,20,0],'-r')

plt.plot(U_vec, Usp_vec, 'o-', label=r'$U_{sp}$', alpha=1, lw=2)
plt.plot(U_vec, Uch_vec, 'o-', label=r'$U_{ch}$', alpha=1, lw=2)

plt.axis('off');

### Note: weakness of RPA

Given the above sum rules, note that in TPSC the following sum-rule, a consequence of the Pauli principle, is satisfied:

\begin{equation}
2\frac{T}{N}\sum_{\mathbf{q},iq_n} \left (\frac{\chi_0(\mathbf{q},iq_n)}{1-\frac{U_{sp}}{2}\chi_0(\mathbf{q},iq_n)}+\frac{\chi_0(\mathbf{q},iq_n)}{1+\frac{U_{ch}}{2}\chi_0(\mathbf{q},iq_n)}\right)=2n-n^2.
\end{equation}

Note that the right-hand side is independent of interactions.

In RPA the left hand of the above equation takes the form

\begin{equation}
\frac{T}{N}\sum_{\mathbf{q},iq_n} \left (\frac{\chi_0(\mathbf{q},iq_n)}{1-\frac{U}{2}\chi_0(\mathbf{q},iq_n)}+\frac{\chi_0(\mathbf{q},iq_n)}{1+\frac{U}{2}\chi_0(\mathbf{q},iq_n)}\right)
\end{equation}

Note that the bare $U$ enters in both denominators. 

Let us show the violation of this sum rule by RPA.

In [ ]:
import numpy as np

n = 1.0
U_vec = np.arange(0., 2.5, 0.25)
sum_chi_vec = np.zeros_like(U_vec)

kmesh, wmesh = chi0_kw.mesh.components

# Printing the header of the table
print ''.join('| %-11s' % s for s in ['U', 'sum_chi', '2n-n*n']), '|'
print '-'*41

# Loop over the different values of U
for idx, U in enumerate(U_vec):
    sum_chi = chi_rpa(chi0_kw, +0.5*U) + chi_rpa(chi0_kw, -0.5*U)
    sum_chi = sum_chi.data.sum() / len(kmesh) / wmesh.beta
    sum_chi_vec[idx] = sum_chi.real
    print ''.join('| %4.4E ' % x for x in [U, sum_chi.real, 2*n-n**2]), '|'

In [ ]:
plt.plot(U_vec, sum_chi_vec, '.-', label=r'Tr[$\chi_{ch} + \chi_{sp}$]')
plt.plot(U_vec, 0*U_vec + 2*n - n**2, 'k-', lw=0.5, label=r'$2n-n^2$')
plt.ylim([0.9, 1.2])
plt.xlabel(r'$U$')
plt.legend()